In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature = 0.1
    , streaming = True
    , callbacks = [StreamingStdOutCallbackHandler()]
)

In [2]:
# 제공하는 예제 -> model이 이런 식으로 답변하면 좋겠다.
examples = [
    {
    "question": "France?",
    "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
    """,
    },
    {
    "question": "Italy?",
    "answer": '''
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
    ''',
    },
    {
    "question": "Greece?",
    "answer": '''
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
    ''',
    },
]

In [3]:
# 예제 형식 지정
example_prompt = ChatPromptTemplate.from_messages(
    [
    ("human", "What do you know about {question}?")
    ,
    ("ai"   , "{answer}")
    ]
)

# 이런식으로 답변하라고 예제 제공
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples       = examples
)

# 위의 예제를 가지고 진짜로 질문을 던질 prmopt 생성
# ("system", ...), ("human", ...) --> 해당 과정이 한 번더 필요하니까 ChatPromptTemplate.from_messages 를 2번 사용한 듯
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert")
        , ("human", "What do you know about {question}?")
        , example_prompt
    ]
)

chain = final_prompt | chat
chain.invoke({"question":"Korea"})

I know this:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korean Won
        Landmarks: The Demilitarized Zone (DMZ), Gyeongbokgung Palace, Jeju Island

AIMessageChunk(content='I know this:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korean Won\n        Landmarks: The Demilitarized Zone (DMZ), Gyeongbokgung Palace, Jeju Island')